In [2]:
import pandas as pd
import os
import sys
import json
from datasets import load_dataset
import random
sys.path.insert(0, '/home/al2644/research/')

# Examples

In [9]:
root = '/share/goyal/lio/knowledge_update/wikipedia/20231101.en'
example_df = pd.read_excel(os.path.join(root, 'excel/sample=300_annotated.xlsx'), engine = 'openpyxl')

In [13]:
examples = example_df.drop_duplicates(subset=['Reason'])

# Input Data

In [3]:
from codebase.wiki_entities_knowledge.PopQA.filter_entities import *

In [139]:
popqa_raw_df = pd.read_csv('/home/al2644/research/datasets/adaptive-retrieval/data/popQA.tsv', sep ='\t')

In [140]:
popqa_raw_df

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[""journo"",""journalists""]",http://www.wikidata.org/entity/Q6247345,http://www.wikidata.org/entity/Q1930187,John Mayne,Journalist,236,24952,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,"[""Eleanor McCutcheon Davis""]","[""graphic artist"",""animator"",""illustrator""]",http://www.wikidata.org/entity/Q5354261,http://www.wikidata.org/entity/Q1114448,Eleanor Davis,Cartoonist,317,9649,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[""Holland"",""the Netherlands"",""NL"",""NED"",""Neder...","[""Mokum"",""Amsterdam, NL"",""Amsterdam, Netherlan...",http://www.wikidata.org/entity/Q55,http://www.wikidata.org/entity/Q727,Netherlands,Amsterdam,416350,147710,What is the capital of Netherlands?,"[""The Hague"", ""Den Haag"", ""'s-Gravenhage"", ""Ha..."
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,"[""Celje city municipality""]",[],http://www.wikidata.org/entity/Q3441823,http://www.wikidata.org/entity/Q1012,City Municipality of Celje,Celje,335,4086,What is the capital of City Municipality of Ce...,"[""Celje""]"
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[""National Capital Region of Delhi"",""NCR""]","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q1353,http://www.wikidata.org/entity/Q987,Delhi,New Delhi,219227,128193,What is the capital of Delhi?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[""occupied Palestinian territories"",""OPT"",""Wes...","[""Yerushalayim"",""J'lem"",""Aelia Capitolina"",""Al...",http://www.wikidata.org/entity/Q407199,http://www.wikidata.org/entity/Q1218,Palestinian territories,Jerusalem,17545,136734,What is the capital of Palestinian territories?,"[""Jerusalem"", ""Yerushalayim"", ""J'lem"", ""Aelia ..."


In [137]:
filename = 'model=meta-llama_Llama-2-7b-chat-hf-input=llama2_7b-method=vanilla-shots=15-n=14267.csv'
popqa_df = load_input_df(filename)

In [5]:
input_df = pd.read_csv(os.path.join(ROOT_PATH, 'model=meta-llama_Llama-2-7b-chat-hf-input=llama2_7b-method=vanilla-shots=15-n=14267.csv')).drop(columns = 'Unnamed: 0')

In [135]:
root = '/share/goyal/lio/knowledge_update/wikipedia/PopQA'
filename = 'step0_input.pickle'
step0_input_df = pd.read_pickle(os.path.join(root, filename))

In [138]:
popqa_df

,popqa_id,title
7,2357044,The Reader
8,4066277,"Philadelphia, New York"
9,1642428,Peter W. Barca
13,1223424,Vienna
14,5999251,Cacheu Region
...,...,...
14243,1941759,Netherlands Antilles
14252,396409,Outfoxed
14264,190883,Huntington Wilson
14265,5649223,The Road (2009 film)


# Select Entities

In [8]:
io_root = '/home/al2644/research/openai_batch_io/wikientities/'
root = '/share/goyal/lio/knowledge_update/wikipedia/PopQA'

In [11]:
'''Change Here'''
dataname = 'popqa_step0'

input_filename = f'{dataname}_input.jsonl'
batch_log_filename = f'{dataname}_batch_log.json'
cache_filename = f'step0_output.pickle'

In [92]:
from codebase.wiki_entities_knowledge.PopQA import step0_select_entities
import importlib; importlib.reload(step0_select_entities)
import openaiAPI

In [1]:
step0_output_df = step0_select_entities.select_entities(df = step0_input_df,
                                            max_tokens=512, batch_size=20,
                                            input_filepath=os.path.join(io_root, input_filename),
                                            batch_log_filepath=os.path.join(io_root, batch_log_filename),
                                            mode='batch_stream'
                                            )

In [25]:
output_df = step0_select_entities.retrieve_outputs(batch_log_filepath=os.path.join(io_root, batch_log_filename),
                                                  cache_filepath=os.path.join(root, cache_filename))

Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: completed
Batch log status: co

Retrieving output file-qFVWPpWKSkgncsY7RXtNB10R
Retrieving output file-w4s9zfJgMAR7AhKAZlWWBobc
Retrieving output file-HFAtjZRekExE1f4cmg87FVhr
Retrieving output file-Ir1dW8JymynOwK9w7nUOML1e
Retrieving output file-I3vXSlfb3DcxtcN8YM1Jb8xr
Retrieving output file-NwhFU1qmRQh350CDOvqovIcg
Retrieving output file-twUIwqXSPlV7I4B19fbeIrFp
Retrieving output file-QCKsyjm1w0j17m3t4nonvq3k
Retrieving output file-qcwe050j9ZdmEs8LB41v8SY3
Retrieving output file-bWneqXWUjMpxThEiAkmaAos0
Retrieving output file-JzkxoYsQ0VFNMywBzAWJStCS
Retrieving output file-HriyWYQYpGDIWkBewRJwTySz
Retrieving output file-DMhgG0kHlV5463IliphtQPnn
Retrieving output file-l3g02Yek4jumifiCvq7Ow67Q
Retrieving output file-EmJL0R9bFD9zr2lC5w3UG78q
Retrieving output file-h1v6muzl6JarzUiOFDFqAT1u
Retrieving output file-6bDBPHSgpVhtlyhp9hSHMwFP
Retrieving output file-ppMkbc8EyGtEOhPLPispgb39
Retrieving output file-nDroHgN2jtgpn7xJNmm2sXWg
Retrieving output file-fTdiLGyUzsMtxFNNIbb2J7Xm
Retrieving output file-BjKTjuLdKvBEBUdfr

In [110]:
step0_output_df = step0_select_entities.postprocess_outputs(step0_input_df, output_df)

In [113]:
step1_input_df = step0_output_df[step0_output_df['step0_output'] == True]
# step1_input_df.to_pickle(os.path.join(root, 'step1_input.pickle'))

In [133]:
idx = 890
print(step1_input_df['title'].iloc[idx])
print(step1_input_df['text'].iloc[idx])

Jimmy Wayne
Jimmy Wayne Barber (born October 23, 1972) is an American country music singer and songwriter. He released his self-titled debut album in 2003 on the DreamWorks Records label. Four singles were released from it, including "Stay Gone" and "I Love You This Much", which both reached Top Ten on the Billboard country charts. A second album, Do You Believe Me Now, was released in August 2008 via Big Machine Records subsidiary Valory Music Group, and its title track became his first Number One hit in late 2008. Sara Smile followed in 2009.

Early life 
Jimmy Wayne was born on October 23, 1972, in Kings Mountain, North Carolina, and grew up in Bessemer City. His biological father abandoned him, and he and his sister were raised in and out of foster homes or were left with other people when their mother would leave them or was in prison. She served four months in prison in 1985 when Wayne was 12. After entering a group home, Wayne ran away and lived with his mother for a brief time 

# Experiment

In [93]:
from openaiAPI import generate_response
from codebase.wiki_entities_knowledge import prompts
from codebase.wiki_entities_knowledge.prompts import *
import importlib; importlib.reload(prompts)

<module 'codebase.wiki_entities_knowledge.prompts' from '/home/al2644/research/codebase/wiki_entities_knowledge/prompts.py'>

In [103]:
entity = 'International Islamic Fiqh Academy'
summary = '''International Islamic Fiqh Academy () is an international Islamic institution for the advanced study of Islamic jurisprudence and law based in Jeddah, Saudi Arabia. It was founded following a resolution by the Third Islamic Summit Conference of the Organization of Islamic Cooperation, held in Mecca, that called for the establishment of an Islamic Fiqh Academy. The International Islamic Fiqh Academy was established in June 1983 as a subsidiary organ of the Organization of Islamic Cooperation. It consists of 57 member states and representatives.'''

In [104]:
system_prompt = prompts.SYSTEM_PROMPT
input_prompt_template = prompts.STEP0_INPUT_PROMPT_TEMPLATE
input_prompt = input_prompt_template.format(entity = entity, summary = summary)

In [106]:
temperature = 0.5
max_tokens = 128
model = 'gpt-4o-mini'
response = generate_response(input_prompt=input_prompt, system_prompt=system_prompt, model=model,
                            temperature=temperature, max_tokens=max_tokens)

In [107]:
print(response)

Entity: International Islamic Fiqh Academy  
Reason: The Academy is currently active and exists in the real world. It is a concrete institution that focuses on Islamic jurisprudence, and while its interpretations and rulings may evolve over time, it is generally considered to provide objective knowledge regarding Islamic law.  
Output: [True]
